In [122]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import psychoanalyze as pa

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
min_intensity = -4
max_intensity = 4
fixed_min = -4
fixed_max = 4
k=1
x_0=0
n_trials = 100
n_blocks = 2
n_subjects = 2

In [146]:
pa.points.from_trials(pa.trials.moc_sample(
    min_intensity, max_intensity, n_trials, k, x_0
))

KeyError: 'Result'

In [142]:
trials = pd.concat(
        {
            day: pd.concat(
                {
                    subj: pd.concat(
                        {
                            fixed_intensity: pa.trials.moc_sample(
                                min_intensity, max_intensity, n_trials, k, x_0
                            )
                            for fixed_intensity in range(fixed_min, fixed_max + 1)
                        },
                        names=["Fixed Intensity"],
                    )
                    for subj in range(n_subjects)
                },
                names=["Subject"],
            )
            for day in range(5)
        },
        names=["Day"],
    )
trials

Day  Subject  Fixed Intensity  Intensity
0    0        -4                0            True
                               -3           False
                               -1           False
                                1            True
                                1            True
                                            ...  
4    1         4                0           False
                               -1           False
                               -2           False
                                0            True
                               -1           False
Name: Result, Length: 9000, dtype: bool

In [149]:
pa.points.n(trials)

Day  Subject  Fixed Intensity
0    0        -4                 100
              -3                 100
              -2                 100
              -1                 100
               0                 100
                                ... 
4    1         0                 100
               1                 100
               2                 100
               3                 100
               4                 100
Name: Result, Length: 90, dtype: int64

In [158]:
pa.points.from_trials(trials)

Hits    n  Hit Rate
Day Subject Fixed Intensity                     
0   0       -4                 58  100      0.58
            -3                 47  100      0.47
            -2                 55  100      0.55
            -1                 50  100      0.50
             0                 58  100      0.58
...                           ...  ...       ...
4   1        0                 46  100      0.46
             1                 45  100      0.45
             2                 48  100      0.48
             3                 49  100      0.49
             4                 58  100      0.58

[90 rows x 3 columns]

In [130]:
pd.concat(
    {
        day: pa.sessions.from_frames_dict(
            {
                subj: pa.trials.moc_sample(
                    min_intensity, max_intensity, n_trials, k, x_0, fixed_min, fixed_max
                )
                for subj in range(n_subjects)
            }
        ) for day in range(5)
    },
    names=["Day"] 
)

ValueError: No objects to concatenate

In [132]:
pa.sessions.from_frames_dict(
            {
                subj: pa.trials.moc_sample(
                    min_intensity, max_intensity, n_trials, k, x_0, fixed_min, fixed_max
                )
                for subj in range(n_subjects)
            }
        ) 

ValueError: No objects to concatenate

In [133]:
pa.trials.moc_sample(
                    min_intensity, max_intensity, n_trials, k, x_0, fixed_min, fixed_max
                )

ValueError: No objects to concatenate

In [114]:
fits = trials.groupby(["Subject", "Block"]).apply(pa.blocks.get_fit)
fits

Subject  Block
0        0        LogisticRegression()
         1        LogisticRegression()
1        0        LogisticRegression()
         1        LogisticRegression()
dtype: object

In [113]:
pa.blocks.make_predictions(fit, intensity_choices=intensity_choices)

Intensity
-4    0.030736
-3    0.074144
-2    0.168215
-1    0.338060
0     0.563266
1     0.765093
2     0.891600
3     0.954068
4     0.981292
Name: Hit Rate, dtype: float64

In [69]:
predictions = pa.subjects.make_predictions(fits, intensity_choices)
predictions

Hit Rate
Block Intensity          
0     -4         0.020199
      -3         0.049223
      -2         0.115058
      -1         0.246149
       0         0.450553
       1         0.673133
       2         0.837972
       3         0.928511
       4         0.970254
1     -4         0.011850
      -3         0.035786
      -2         0.103026
      -1         0.262244
       0         0.523824
       1         0.772956
       2         0.913314
       3         0.970243
       4         0.990187

In [ ]:
predictions = fits.apply(pa.blocks.make_predictions, intensity_choices=intensity_choices)
predictions

In [121]:
params = fits.apply(pa.blocks.fit_params)
params

slope  intercept
Subject Block                     
0       0      0.926379   0.254427
        1      1.599937   0.134766
1       0      1.389645  -0.099479
        1      0.729999  -0.419109

In [59]:
pd.concat(
    {"slope": pd.Series({
        block: fits[block].coef_[0][0] for block in fits
    }),
    "intercept": pd.Series({
        block: fits[block].intercept_[0] for block in fits
    })
    }, names=["param", "Block"])

param      Block
slope      0        1.075417
           1        0.856080
intercept  0       -0.095133
           1        0.045637
dtype: float64